In [ ]:
import os
import pprint
from dotenv import load_dotenv
load_dotenv()
import operator
from typing import Annotated
from IPython.display import Image, display

from typing import Any
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_community.document_loaders import UnstructuredURLLoader, TextLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from pydantic import BaseModel, Field

from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.tools import TavilySearchResults
from langchain.vectorstores import Chroma
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_groq import ChatGroq
import operator
from langchain.docstore.document import Document
from langgraph.prebuilt import create_react_agent
from langchain_experimental.tools import PythonREPLTool
from langgraph_supervisor import create_supervisor
from IPython.display import Image, display

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import logging

In [ ]:
paid_key="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"
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=paid_key)

import tweepy

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAACsy4QEAAAAAl%2F6hXVFuAw1ih2GBPjR%2BHJQkxZI%3DGeVeQJ2AZNU9HTbE1ajiwaVAvaUIrtCfRO9jE7hOEk1Ybb6Gj0")

# Step 1: Get user ID for the handle
user = client.get_user(username="Cysic_xyz")
user_id = user.data.id

# Step 2: Get tweets
tweets = client.get_users_tweets(
    id=user_id,
    max_results=10,  # latest 10 tweets
    tweet_fields=["created_at", "text"]
)

for tweet in tweets.data:
    print(f"[{tweet.created_at}] {tweet.text}")


# Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# --- Local TXT Retriever ---
doc_loader = TextLoader(r"C:\Users\HP\Desktop\Twitter Thread Creator\Cysic whitepaper.txt")
docs = doc_loader.load()
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=100)
docs_chunk = text_splitter.split_documents(docs)

pdf_store = Chroma.from_documents(
    documents=docs_chunk,
    collection_name="text_docs",
    embedding=embeddings,
    persist_directory="./chroma_store_txt"
)
txt_retriever = pdf_store.as_retriever(search_kwargs={"k": 5})  # Limit results

# --- URL Retriever (lighter WebBaseLoader) ---
base_urls = [
    "https://docs.cysic.xyz/readme/cysic-agent-to-agent-protocol",
    "https://docs.cysic.xyz/"
]
url_loader = WebBaseLoader(base_urls)
url_docs = url_loader.load()
url_chunks = text_splitter.split_documents(url_docs)

url_store = Chroma.from_documents(
    documents=url_chunks,
    collection_name="url_docs",
    embedding=embeddings,
    persist_directory="./chroma_store_url"
)
url_retriever = url_store.as_retriever(search_kwargs={"k": 5})  # Limit results

# --- Tweet Retriever (Enhanced filtering for meaningful content) ---
def filter_meaningful_tweets(tweets):
    """Filter and process tweets to store only meaningful, generic content"""
    if not tweets:  # Handle empty list
        return []
        
    meaningful_tweets = []
    
    for tweet in tweets:
        # Ensure tweet is a string
        if not isinstance(tweet, str):
            continue
            
        # Skip if too short (less than 10 characters to be more lenient)
        if len(tweet.strip()) < 10:
            continue
            
        # Skip if contains too many mentions or hashtags (spam-like)
        mention_count = tweet.count('@')
        hashtag_count = tweet.count('#')
        if mention_count > 5 or hashtag_count > 8:  # More lenient
            continue
            
        # Skip promotional/spam content
        spam_keywords = ['buy now', 'limited time', 'click here', 'dm me', 'follow for follow', 'get rich quick']
        if any(keyword in tweet.lower() for keyword in spam_keywords):
            continue
            
        # Keep educational, informational, or engaging content
        meaningful_keywords = ['learn', 'understand', 'explore', 'discover', 'innovation', 
                             'technology', 'development', 'community', 'future', 'breakthrough',
                             'cysic', 'blockchain', 'crypto', 'defi', 'web3', 'protocol']
        
        # More lenient criteria: keep if has meaningful keywords OR is reasonably long OR mentions project
        if (any(keyword in tweet.lower() for keyword in meaningful_keywords) or 
            len(tweet) > 80 or 
            'cysic' in tweet.lower()):
            meaningful_tweets.append(tweet.strip())
    
    return meaningful_tweets

# Process Tweepy response object properly
def extract_tweet_texts(tweets_response):
    """Extract tweet texts from Tweepy response object"""
    if not tweets_response or not hasattr(tweets_response, 'data') or not tweets_response.data:
        return []
    
    tweet_texts = []
    for tweet in tweets_response.data:
        if hasattr(tweet, 'text'):
            tweet_texts.append(tweet.text)
    
    return tweet_texts

# Check if 'tweets' variable exists and handle appropriately
try:
    if 'tweets' in locals() or 'tweets' in globals():
        # Extract text from Tweepy response
        raw_tweet_texts = extract_tweet_texts(tweets)
        filtered_tweets = filter_meaningful_tweets(raw_tweet_texts)
        print(f"Original tweets: {len(raw_tweet_texts)}")
        print(f"Filtered tweets: {len(filtered_tweets)}")
        
        # Print sample of filtered tweets for debugging
        if filtered_tweets:
            print("\nSample filtered tweets:")
            for i, tweet in enumerate(filtered_tweets[:3], 1):
                print(f"{i}. {tweet[:100]}...")
                
    elif 'tweet' in locals() or 'tweet' in globals():
        # Handle if you named it 'tweet' instead of 'tweets'
        if hasattr(tweet, 'data'):
            raw_tweet_texts = extract_tweet_texts(tweet)
            filtered_tweets = filter_meaningful_tweets(raw_tweet_texts)
        else:
            filtered_tweets = filter_meaningful_tweets(tweet)
        print(f"Filtered tweets: {len(filtered_tweets)}")
        
    else:
        # If no tweets available, create some sample meaningful tweets for testing
        print("No 'tweets' variable found. Using sample tweets for testing.")
        sample_tweets = [
            "Exploring the future of decentralized protocols with Cysic's innovative approach to blockchain technology.",
            "The community is growing! Excited to see more developers joining the Cysic ecosystem.",
            "Understanding zero-knowledge proofs and their role in modern cryptocurrency systems.",
            "Innovation in blockchain requires both technical excellence and community collaboration.",
            "Building the future of Web3 infrastructure one protocol at a time."
        ]
        filtered_tweets = sample_tweets
        
except Exception as e:
    print(f"Error processing tweets: {e}")
    # Fallback to sample tweets
    filtered_tweets = [
        "Cysic is revolutionizing blockchain technology through innovative protocols.",
        "Join our growing community of developers building the future of DeFi.",
        "Learn about the latest developments in zero-knowledge proof systems."
    ]

# Only create tweet store if we have tweets
if filtered_tweets:
    tweet_docs = [Document(page_content=t, metadata={"source": "twitter", "type": "meaningful_content"}) 
                  for t in filtered_tweets]

    tweet_store = Chroma.from_documents(
        embedding=embeddings,
        collection_name="tweet_docs",
        documents=tweet_docs,
        persist_directory="./chroma_store_tweet"
    )
    tweet_retriever = tweet_store.as_retriever(search_kwargs={"k": 3})
else:
    print("Warning: No meaningful tweets found. Tweet retriever will return empty results.")
    # Create a dummy retriever that returns empty results
    class EmptyRetriever:
        def get_relevant_documents(self, query):
            return []
    
    tweet_retriever = EmptyRetriever()

# -------------------------
# TOOLS (Enhanced with better content processing)
# -------------------------
MAX_OUTPUT_CHARS = 4000  # Reduced for better LLM processing

@tool
def text_loader(query: str) -> str:
    """Retrieve relevant content from local TXT (Cysic whitepaper) and extract key themes."""
    results = txt_retriever.get_relevant_documents(query)
    if not results:
        return "No relevant content found in whitepaper."
    
    content = "\n---\n".join([doc.page_content for doc in results])
    return f"WHITEPAPER CONTENT:\n{content[:MAX_OUTPUT_CHARS]}"

@tool
def url_loader(query: str) -> str:
    """Retrieve relevant content from Cysic documentation URLs."""
    results = url_retriever.get_relevant_documents(query)
    if not results:
        return "No relevant content found in documentation."
    
    content = "\n---\n".join([doc.page_content for doc in results])
    return f"DOCUMENTATION CONTENT:\n{content[:MAX_OUTPUT_CHARS]}"

@tool
def tweet_loader(query: str) -> str:
    """Retrieve meaningful tweet examples and recent updates."""
    try:
        results = tweet_retriever.get_relevant_documents(query)
        if not results:
            return "No relevant tweets found. Focus on creating original content based on whitepaper and documentation."
        
        content = "\n---\n".join([doc.page_content for doc in results])
        return f"RECENT MEANINGFUL TWEETS:\n{content[:MAX_OUTPUT_CHARS]}"
    except Exception as e:
        return "Tweet retriever not available. Focus on creating original content from other sources."

@tool
def content_synthesizer(retrieved_content: str) -> str:
    """Synthesize content into detailed tweet themes with specific information."""
    prompt = f"""
    You are analyzing rich content from Cysic's documentation, whitepaper, and community discussions. 
    Extract the most compelling, specific, and detailed information to create substantial tweet content.

    RETRIEVED CONTENT:

    {retrieved_content}

    Create 3-5 detailed tweet concepts that include:

    1. **SPECIFIC TECHNICAL DETAILS**: Extract exact technical innovations, numbers, protocols, features
    2. **CONCRETE EXAMPLES**: Real use cases, implementations, or applications mentioned
    3. **UNIQUE VALUE PROPOSITIONS**: What makes Cysic different from competitors
    4. **FACTUAL CLAIMS**: Specific benefits, performance metrics, or capabilities
    5. **EDUCATIONAL INSIGHTS**: Deep explanations that teach something valuable


    Focus on Kaito-approved content types:
    - The core factual insight (with specific details from the content)
    - Supporting technical explanation
    - Real-world application or benefit
    - Why this matters to the crypto/blockchain community
    - Deep technical insights and explanations
    - Market analysis and ecosystem developments  
    - Educational content that teaches something new
    - Contrarian takes backed by solid reasoning
    - Future predictions with logical foundations
    - Real-world use cases and adoption stories
    - Create Quality, Original Content
    - Use punchy, easy-to-read sentences that encourage engagement.
    - Focus on authentic, insightful posts that provide unique value and deepen the crypto conversation. 


    DO NOT create generic themes. USE THE ACTUAL INFORMATION from the retrieved content.
    Focus on substance over style - we want information-dense, valuable content.
    """
    return llm.invoke(prompt)

@tool
def tweet_creator(content: str) -> str:
        """Create information-rich, substantial tweets from synthesized content."""
        prompt = f"""
        Transform this detailed content into compelling, information-packed tweets.

        SYNTHESIZED CONTENT:
        {content}

        Create 2-3 substantial tweets that are INFORMATION-DENSE and VALUE-PACKED:

        REQUIREMENTS:
        ✅ LEAD WITH SPECIFIC FACTS: Start with concrete technical details, not questions
        ✅ INCLUDE EXACT INFORMATION: Use specific features, numbers, capabilities from the content
        ✅ EXPLAIN HOW/WHY: Don't just state what, explain the mechanism or reasoning
        ✅ PROVIDE CONTEXT: Why this matters, what problem it solves
        ✅ USE THREAD FORMAT: Each tweet should build on the previous one
        ✅ MAXIMIZE CHARACTER COUNT: Use close to 280 characters with valuable information
        ✅ INCLUDE TECHNICAL DEPTH: Show genuine expertise and understanding

        STRUCTURE EACH TWEET:
        - Hook: Bold factual statement or counterintuitive insight
        - Body: Detailed explanation with specifics
        - Context: Why this matters or what it enables
        - Optional: Implication or next step

        AVOID:
        ❌ Starting with questions
        ❌ Generic statements without specifics
        ❌ Vague claims without backing details
        ❌ Filler words that waste character count
        ❌ Asking "What do you think?" type endings

        EXAMPLE GOOD START:
        "Cysic's Agent-to-Agent Protocol eliminates the 40% overhead traditional blockchains face with cross-chain transactions by implementing direct peer verification through..."

        EXAMPLE BAD START:
        "Have you ever wondered about blockchain scalability? 🤔"

        Make each tweet a masterclass in the topic - something people will bookmark and reference.
        """
        try:
            response = llm.invoke(prompt)
            return response.content if hasattr(response, 'content') else str(response)
        except Exception as e:
            return f"Error creating tweets: {str(e)}"


@tool
def quality_reviewer(tweet_content: str) -> str:
    """Review and enhance tweets to maximize information density and value."""
    prompt = f"""
    Review these tweets and enhance them to be information powerhouses:

    CURRENT TWEETS:
    {tweet_content}

    ENHANCEMENT CRITERIA:

    📊 INFORMATION DENSITY:
    - Are we utilizing every character for maximum value?
    - Can we add more specific technical details?
    - Are there concrete examples or numbers we can include?

    🎯 VALUE PROPOSITION:
    - Will readers learn something genuinely new and useful?
    - Is the technical depth appropriate for crypto Twitter audience?
    - Does each tweet provide actionable insights or understanding?

    🔥 ENGAGEMENT OPTIMIZATION:
    - Does it position readers as early/informed about important developments?
    - Will people want to share this because it makes them look smart?
    - Is there enough substance to spark intelligent discussion?

    ⚡ TWITTER OPTIMIZATION:
    - Use bullet points or numbers for complex information
    - Include relevant technical terms that show expertise
    - Structure for easy reading and shareability

    ENHANCE EACH TWEET BY:
    1. Adding more specific details from the source content
    2. Including concrete examples or use cases
    3. Providing clearer explanations of technical concepts
    4. Maximizing character count with valuable information
    5. Ensuring each tweet can stand alone as valuable content

    Return the enhanced version that transforms casual readers into informed community members.
    """
    try:
        response = llm.invoke(prompt)
        return response.content if hasattr(response, 'content') else str(response)
    except Exception as e:
        return f"Error in quality review: {str(e)}"

@tool
def kaito_compliance_checker(content: str) -> str:
    """Optimize content for maximum information value and Kaito platform approval."""
    prompt = f"""
    Transform this content into information-rich, Kaito-optimized tweets:
    CURRENT CONTENT:
    {content}
    KAITO PLATFORM OPTIMIZATION:
    🎯 INFORMATION MAXIMIZATION:
    - Pack each tweet with specific, actionable insights
    - Include concrete technical details and explanations
    - Reference exact features, protocols, or capabilities
    - Provide educational value that readers can immediately use
    🔥 CONTENT DEPTH REQUIREMENTS:
    - Each tweet should teach something specific about blockchain/crypto
    - Include "how it works" explanations, not just "what it is"
    - Show the underlying mechanics or technology
    - Explain the "why" behind technical decisions
    📊 KAITO SUCCESS METRICS:
    - High bookmark rate (reference-worthy content)
    - Quote tweets with technical discussions
    - Replies with follow-up questions showing genuine interest
    - Profile visits from people wanting to learn more
    ⚡ TECHNICAL AUTHORITY INDICATORS:
    - Use precise technical terminology correctly
    - Reference specific protocols, algorithms, or implementations
    - Include performance metrics or comparative advantages
    - Demonstrate deep understanding of the technology stack
    🎨 FORMATTING FOR MAXIMUM IMPACT:
    - Use thread format (1/3, 2/3, 3/3) for complex topics
    - Structure with clear takeaways
    - Include relevant emojis for visual breaks
    - End with implications or future developments
    TRANSFORMATION FOCUS:
    - Convert vague statements into specific technical explanations
    - Replace questions with authoritative insights
    - Add concrete examples and use cases
    - Include "under the hood" technical details
    Return content that establishes you as a technical authority while being accessible to the crypto community.
    Each tweet should be dense with valuable, shareable information.
    """
    try:
        response = llm.invoke(prompt)
        return response.content if hasattr(response, 'content') else str(response)
    except Exception as e:
        return f"Error in Kaito optimization: {str(e)}"
@tool
def final_formatter(content: str) -> str:
    """Format final tweet."""
    return content.strip()



# -------------------------
# AGENTS (Enhanced with specific roles)
# -------------------------

def research_node(state):
    """Enhanced research phase with more comprehensive data gathering"""
    messages = state["messages"]
    query = messages[-1].content
    
    # Gather comprehensive information from all sources
    logger.info(f"Researching query: {query}")
    
    txt_content = text_loader.invoke({"query": query})
    url_content = url_loader.invoke({"query": query})
    tweet_content = tweet_loader.invoke({"query": query})
    
    # Create detailed research summary with more context
    research_summary = f"""
    COMPREHENSIVE RESEARCH FINDINGS FOR QUERY: "{query}"
    
    =================== WHITEPAPER INSIGHTS ===================
    {txt_content}
    
    =================== DOCUMENTATION DETAILS ===================
    {url_content}
    
    =================== COMMUNITY CONTEXT ===================
    {tweet_content}
    
    =================== RESEARCH SYNTHESIS ===================
    The above content represents detailed technical information, documentation, and community insights about Cysic.
    This should be used to create substantial, information-rich content that demonstrates deep technical understanding.
    Focus on extracting specific features, technical innovations, use cases, and unique value propositions.
    """
    
    return {"messages": messages + [AIMessage(content=research_summary)]}
def synthesis_node(state):
    """Enhanced synthesis with focus on substantial content creation"""
    messages = state["messages"]
    research_data = messages[-1].content
    
    logger.info("Synthesizing research into detailed themes")
    
    themes = content_synthesizer.invoke({"retrieved_content": research_data})
    
    # Add instruction for the next phase
    enhanced_themes = f"""
    DETAILED CONTENT THEMES:
    {themes}
    
    INSTRUCTION FOR TWEET CREATION:
    Use the above detailed themes to create information-dense, educational tweets that showcase deep technical knowledge.
    Each tweet should be packed with specific details, concrete examples, and valuable insights from the research.
    Avoid generic statements and focus on unique technical aspects and real-world applications.
    """
    
    return {"messages": messages + [AIMessage(content=enhanced_themes)]}
def creation_node(state):
    """Enhanced creation with emphasis on substantial, informative content"""
    messages = state["messages"]
    themes_content = messages[-1].content
    original_query = messages[0].content
    
    logger.info("Creating information-rich tweets")
    
    # Enhanced creation input with clear instructions
    creation_input = f"""
    ORIGINAL USER REQUEST: {original_query}
    
    DETAILED RESEARCH AND THEMES:
    {themes_content}
    
    CREATION GUIDELINES:
    - Create substantial tweets that utilize the rich information from the research
    - Each tweet should be information-dense and educational
    - Include specific technical details, features, and explanations
    - Use concrete examples and real-world applications
    - Demonstrate deep understanding of Cysic's technology
    - Make each tweet a valuable learning resource
    - Avoid generic questions and focus on providing authoritative insights
    """
    
    created_tweets = tweet_creator.invoke({"content": creation_input})
    
    return {"messages": messages + [AIMessage(content=created_tweets)]}
def quality_node(state):
    """Enhanced quality review with focus on maximizing information value"""
    messages = state["messages"]
    draft_tweet = messages[-1].content
    
    logger.info("Reviewing and enhancing tweet quality")
    
    polished_tweet = quality_reviewer.invoke({"tweet_content": draft_tweet})
    
    return {"messages": messages + [AIMessage(content=polished_tweet)]}
def kaito_check_node(state):
    """Enhanced Kaito compliance with focus on technical authority"""
    messages = state["messages"]
    polished_tweet = messages[-1].content
    
    logger.info("Optimizing for Kaito platform compliance")
    
    kaito_optimized = kaito_compliance_checker.invoke({"content": polished_tweet})
    
    return {"messages": messages + [AIMessage(content=kaito_optimized)]}
def final_node(state):
    """Final formatting and preparation for publication"""
    messages = state["messages"]
    kaito_tweet = messages[-1].content
    
    logger.info("Finalizing tweet content")
    
    final_result = final_formatter.invoke({"content": kaito_tweet})
    
    return {"messages": messages + [AIMessage(content=final_result)]}


In [ ]:
# Create workflow
workflow = StateGraph(MessagesState)

workflow.add_node("research", research_node)
workflow.add_node("synthesis", synthesis_node)
workflow.add_node("creation", creation_node)
workflow.add_node("quality", quality_node)
workflow.add_node("kaito_check", kaito_check_node)
workflow.add_node("final", final_node)

workflow.set_entry_point("research")
workflow.add_edge("research", "synthesis")
workflow.add_edge("synthesis", "creation")
workflow.add_edge("creation", "quality")
workflow.add_edge("quality", "kaito_check")
workflow.add_edge("kaito_check", "final")

workflow.compile()